In [1]:
from eunjeon import Mecab
from openpyxl import load_workbook
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import math


load_wb = load_workbook("C:/Users/HanYong/Desktop/Python/new/LgCNS.xlsx", data_only=True)
load_ws = load_wb['sheet1']
text1 = ""

#셀 좌표로 값 출력
for i in range(2, 223):
    label = load_ws.cell(i,5).value
    
    if label == 1 :
        titles = load_ws.cell(i,3).value 
        contents = load_ws.cell(i,4).value
        text1 += (titles + contents + "\n")
        
#print(text1)

def onlyNouns(article) :
    result = ""
    mecab = Mecab()
    sample = mecab.pos(article)
    for x in sample:
        if 'NNG' in x or 'NNP' in x or 'SL' in x or 'NP' in x :
            result += (x[0]) + ' '
            #print(result)
    return result

text1 = text1.split("\n")
#print(text1)
li_noun = []

for i in text1 :
    i = onlyNouns(i)
    li_noun.append(i)
#print(li_noun)
text1 = "\n".join(li_noun)

def get_doc(sent) :
    """
    이 함수는 텍스트를 문장으로 나누고 각 문장을 document로 
    여겨서 각 document의 총 단어 등장 수를 카운트
    """
    doc_info = []
    i = 0
    for x in sent :
        i += 1
        count = count_words(x)
        temp = {'doc_id' : i, 'doc_length' : count}
        doc_info.append(temp)
    return doc_info

def count_words(sent) :
    """
    이 함수는 각 input 텍스트의 총 단어수를 리턴
    """
    count = 0
    words = krword_tokenize(sent)
    #print(words)
    for word in words :
        count += 1
    return count

def krword_tokenize(sent):
    result = []
    mecab = Mecab()
    sample = mecab.pos(sent)
    for x in sample:
        if 'NNG' in x or 'NNP' in x or 'SL' in x or 'NP' in x :
            result.append(x)
    return result

def create_freq_dict(sents) :
    """
    이 함수는 각 사전의 단어들에 대한 빈도수를 dictionary로 만듬
    """
    i = 0
    freqDict_list = []
    for sent in sents :
        i += 1
        freq_dict = {}
        words = word_tokenize(sent)
        for word in words :
            word = word.lower()
            if word in freq_dict :
                freq_dict[word] += 1
            else :
                freq_dict[word] = 1
            temp = {'doc_id' : i, 'freq_dict' : freq_dict}
        freqDict_list.append(temp)
    return freqDict_list

def computeTF(doc_info, freqDict_list) :
    """
    tf = (document 내에서 단어 등장 빈도 / document내의 전체 단어 수)
    """
    
    TF_scores = []
    for tempDict in freqDict_list :
        id = tempDict['doc_id']
        for k in tempDict['freq_dict'] :
          
            temp = {'doc_id' : id,
                   'TF_score' : tempDict['freq_dict'][k]/doc_info[id-1]['doc_length'],
                   'key' : k}
            TF_scores.append(temp)
    return TF_scores

def computeIDF(doc_info, freqDict_list) :
    """
    idf = log(전체 문서 수/ 단어가 드 안에 들어있는 문서 수)
    """
    IDF_scores = []
    counter = 0
    for dict in freqDict_list :
        counter += 1
        for k in dict['freq_dict'].keys() :
            count = sum([k in tempDict['freq_dict'] for tempDict in freqDict_list])
            temp = {'doc_id' : counter, 'IDF_score' : math.log(len(doc_info)/count), 'key' : k}
            
            IDF_scores.append(temp)
        
    return IDF_scores

def computeTFIDF(TF_scores, IDF_scores) :
    TFIDF_scores = []
    for j in IDF_scores :
        for i in TF_scores :
            if j['key'] == i['key'] and j ['doc_id'] == i['doc_id'] :
                temp = {'doc_id' : j['doc_id'],
                       'TFIDF_score' : j['IDF_score']*i['TF_score'],
                       'key' : i['key']}
        TFIDF_scores.append(temp)
    return TFIDF_scores

def remove_string_special_characters(s) :
    """
    s : 문자열
    """
    stripped = re.sub('[^\w\s]', '', s)
    #문자열 중간 공백 삭제
    stripped = re.sub('\s+', ' ', stripped)
    #처음과 끝 공백 삭제
    stripped = stripped.strip()
    
    return stripped

text_sents = text1.split("\n")
doc_info = get_doc(text_sents)

freqDict_list = create_freq_dict(text_sents)
#print(freqDict_list)
TF_scores = computeTF(doc_info, freqDict_list)
IDF_scores = computeIDF(doc_info, freqDict_list)

TFIDF_scores = computeTFIDF(TF_scores, IDF_scores)
#print(TFIDF_scores)

li = []
for i in TFIDF_scores :
    if i['TFIDF_score'] not in li :
        li.append(i['TFIDF_score'])
li.sort()
li.reverse()
li = li[0:100]
#print(li)

for i in li :
    for j in TFIDF_scores :
        if i == j['TFIDF_score'] :
            print(j['key'],j['TFIDF_score'])
            break

브라질 0.3209869603630467
괌 0.3004594110099321
코딩 0.287134469235563
전력 0.28017363361773123
편의점 0.26417173482066425
코딩 0.25124266058111766
엑스레이 0.23245352916681386
스마일 0.22339751820138778
한전 0.21774155112491114
태양광 0.21371670976802273
에이 0.21324308555587015
오롯 0.20221327078573895
nc 0.1962160711198501
골프장 0.19153057253505143
꽃 0.19140322507990845
담합 0.18954940938299572
발전소 0.1867824224118208
학원 0.1851847848248346
명회 0.18510188433653696
발전소 0.18454550717335588
u 0.18312369625965794
프리미어 0.18277978761931726
앤 0.18173639553041812
창원시 0.17991903157511394
타운 0.17592554558359288
출입 0.17375362526774604
소설 0.17304152024615693
점주 0.1724760250819538
nbp 0.17215955531116123
노 0.16754813865104082
정부 0.16684862260777064
태양광 0.16639016564689882
단비 0.16600725448493964
시티 0.1655578170756999
전무 0.16341891268891848
ge 0.16240273643143746
페이 0.15738350100520251
티 0.15565201867651735
대회 0.1546265464733944
워즈 0.15231648968276437
도시 0.14964739829357546
미 0.14942593792945666
구글 0.14892549411888137
교육 0.148370359

In [3]:
from eunjeon import Mecab
from openpyxl import load_workbook
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math
import re


load_wb = load_workbook("C:/Users/HanYong/Desktop/Python/new/LgCNS.xlsx", data_only=True)
load_ws = load_wb['sheet1']
wordDict = {}

#셀 좌표로 값 출력
for i in range(2, 223):
    label = load_ws.cell(i,5).value
    
    if label == 1 :
        titles = load_ws.cell(i,3).value 
        contents = load_ws.cell(i,4).value

        texts = titles + contents
        mecab = Mecab()
        malist = mecab.pos(texts)
        for x in malist:
            if 'NNG' in x or 'NNP' in x or 'SL' in x or 'NP' in x :
                keyword = x[0]
                if keyword in wordDict:
                    wordDict[keyword] += 1
                else:
                    wordDict[keyword] = 1

wordDict = sorted(wordDict.items(), key=lambda t : t[1], reverse=True)

print(wordDict)

[('LG', 777), ('CNS', 504), ('기술', 418), ('기업', 361), ('서비스', 361), ('클라우드', 307), ('사업', 277), ('블록체인', 243), ('AI', 241), ('데이터', 241), ('스마트', 192), ('시스템', 189), ('국내', 165), ('정보', 157), ('금융', 157), ('IT', 153), ('시장', 141), ('플랫폼', 138), ('개발', 134), ('구축', 127), ('제공', 126), ('말', 122), ('인공지능', 120), ('산업', 114), ('빅', 114), ('전자', 113), ('분석', 107), ('h', 105), ('공장', 104), ('한국', 100), ('업체', 97), ('계획', 96), ('분야', 95), ('봇', 94), ('로봇', 91), ('적용', 90), ('글로벌', 89), ('디지털', 89), ('활용', 86), ('운영', 86), ('기반', 85), ('미국', 75), ('고객', 75), ('관리', 74), ('세계', 73), ('투자', 73), ('생산', 71), ('확대', 70), ('도입', 69), ('스타트업', 68), ('제품', 67), ('AWS', 67), ('업무', 66), ('혁신', 64), ('가능', 64), ('올해', 64), ('전환', 64), ('시티', 64), ('소프트웨어', 62), ('솔루션', 61), ('IoT', 61), ('사용', 60), ('해외', 59), ('이', 59), ('계열사', 59), ('회사', 58), ('협력', 58), ('그룹', 58), ('통신', 57), ('팩토리', 57), ('인터넷', 56), ('가상', 56), ('화폐', 56), ('지난해', 55), ('관계자', 55), ('규모', 55), ('구글', 54), ('진행', 54), ('역량', 53),

In [ ]:
res = [i for i in TFIDF_scores if not (i['TFIDF_score'] > 0.01)]

TF_IDF_Minus = []
for i in res :
    if i['key'] in TF_IDF_Minus :
        continue
    TF_IDF_Minus.append(i['key'])

#print(TF_IDF_Minus)

In [6]:
TF_IDF_Minus.remove('cloud')
TF_IDF_Minus.remove('클라우드')
TF_IDF_Minus.remove('반도체')
TF_IDF_Minus.remove('스피커')
TF_IDF_Minus.remove('금융')
TF_IDF_Minus.remove('넥스')
TF_IDF_Minus.remove('레저')
TF_IDF_Minus.remove('nexledger')
TF_IDF_Minus.remove('blockchain')
TF_IDF_Minus.remove('블록')
TF_IDF_Minus.remove('블록체인')


load_wb = load_workbook("C:/Users/HanYong/Desktop/Python/new/LgCNS.xlsx", data_only=True)
load_ws = load_wb['sheet1']
wordList = []

#셀 좌표로 값 출력
for i in range(2, 223):
    label = load_ws.cell(i,5).value
    
    if label == 1 :
        titles = load_ws.cell(i,3).value 
        contents = load_ws.cell(i,4).value


        texts = titles + contents
        
        mecab = Mecab()
        malist = mecab.pos(texts)
        for x in malist:
            if 'NNG' in x or 'NNP' in x or 'SL' in x or 'NP' in x :
                keyword = x[0]
                wordList.append(keyword)
                
for i in TF_IDF_Minus :
    for j in wordList :
        if i == j :
            wordList.remove(j)
            
                
wordFinal = " ".join(wordList)
 
print(wordFinal)

AI IT AI AI SenseTime LG AI AI 스피커 AI IT AI AI AI AI 속국 AI AI IITP AI AI SPRi SPRi AI 상당수 AI SK CC IBM IBM AI SK MS AI LG CNS AI 금융 SAS AI 상당수 AI AI D M H AI AI AI 애플 Siri Bixby AI AI 금융 AI AI AI AI AI 속국 의존 MS OS PC IT AI AI AI 의존 애플 AI 금융 AI AI AI 속국 식민지 틱톡 tiktok 성인 약 틱톡 바이트 댄스 틱톡 약관 법령 명기 연락처 주권 主權 싱크탱크 피터슨 틱톡 미군 병사 스파이 용도 연방 FTC 틱톡 아동 불법 과징금 부과 AI 진입 주권 전쟁 전통 주권 영토 영공 영해 AI 국경 안드로이드 AI 자국 관할 법률 테러 범죄 권한 한국인 제외 자국 김상배 외교 주권 침해 AI 강국 식민지 AI 주권 경계 網 주권 보호법 유튜브 틱톡 세금 특혜 주권 클라우드 클라우드 클라우드 LG 쿠팡 민족 넥슨 왓 챠 컬리 클라우드 금융 우리나라 클라우드 춘천 KT LG SDS LG CNS SK CC IT 고위 주권 최소한 미지수 LG CNS AWS 아태 AWS LG CNS AWS 아태 수여 AWS 클라우드 아세안 ASEAN 호주 수여 LG CNS AWS APN AWS APN 어드밴스 드 APN ISV SI AWS LG CNS AWS 클라우드 IT AWS 클라우드 LG CNS AWS 클라우드 AWS 금융 클라우드 KFC 클라우드 최문근 LG CNS 클라우드 클라우드 클라우드 결과물 전재규 AWS 코리아 LG CNS SI 클라우드 항공사 IT 클라우드 클라우드 AWS LG CNS 블록체인 페북 뉴스룸 블록체인 첫발 시물 갤럭시 S 시리즈 블록체인 키 스토어 先 콜드 월렛 갤럭시 S 지인 물건 원철 블록체인 풍요 갤럭시 S 블록체인 카카오 라쿠텐 IT 블록체인 블록체인 광풍 狂風 블록체인 실생활 세력 놀이터 등락 안정세 블록체인 카카오 블록체인 카카오 자 X 클레이튼 Klaytn 블